In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
from torch.utils.data import TensorDataset
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights   
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score

batch_size = 256
num_workers = 8

In [3]:

# Load the saved fold data from the .pkl file
fold_data = torch.load('/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/cross_val_data/fold_data.pt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = '/home/jaiswalsuraj/suraj_work/Brick-Kilns-Project/data/models/'

# Lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold_info in fold_data:
    fold = fold_info['fold']
    print("Fold: ", fold)
    X_train = fold_info['X_train']
    y_train = fold_info['y_train']
    X_test = fold_info['X_test']
    y_test = fold_info['y_test']
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoader for training and testing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create and train the model
    #print datatype of trainloader
    print("trainloader datatype: ", train_loader.dataset.tensors[1].dtype)
    print("testloader datatype: ", test_loader.dataset.tensors[0].dtype)
    train_model = EfficientNetClassifier    (
        models.efficientnet_b0, EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
    ).to(device)

    iter_losses, epoch_losses = train_fn(
        train_model,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        lr=3e-4,
        epochs=100,
        verbose=True,
        wandb_log=False,
    )
    torch.save(train_model.state_dict(), save_path + f'efficientnet_train_bang_fold_{fold}.pth')
    
    # Evaluate the model on the test set
    with torch.no_grad():
        pred_classes = train_model.predict_class(
            dataloader=test_loader, batch_size=batch_size, verbose=True
        ).to(device)

    test_labels = y_test.to(device)
    # Calculate and print metrics for each fold
    
    accuracy = accuracy_score(pred_classes,test_labels)
    precision = precision_score( pred_classes,test_labels)
    recall = recall_score( pred_classes,test_labels)
    f1 = f1_score( pred_classes,test_labels)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\n")
    # Calculate and store metrics for each fold
    accuracy_list.append(accuracy_score( pred_classes,test_labels))
    precision_list.append(precision_score( pred_classes,test_labels))
    recall_list.append(recall_score( pred_classes,test_labels))
    f1_list.append(f1_score( pred_classes,test_labels))

# Calculate and print the mean of metrics across all folds
print("Mean Accuracy: ", sum(accuracy_list) / len(accuracy_list))
print("Mean Precision: ", sum(precision_list) / len(precision_list))
print("Mean Recall: ", sum(recall_list) / len(recall_list))
print("Mean F1: ", sum(f1_list) / len(f1_list))


Fold:  1
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


/home/jaiswalsuraj/miniconda3/envs/torch_gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 25/25 [00:02<00:00,  8.81it/s]


Accuracy: 0.9876
Precision: 0.8929
Recall: 0.9245
F1 Score: 0.9085


Fold:  2
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


100%|██████████| 25/25 [00:02<00:00,  9.23it/s]


Accuracy: 0.9873
Precision: 0.9113
Recall: 0.8962
F1 Score: 0.9037


Fold:  3
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


Loss: 0.00265273:  83%|████████▎ | 83/100 [39:47<08:02, 28.36s/it]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
print("Classification Report:\n", classification_report(test_labels.cpu().numpy(), pred_classes.cpu().numpy()))

In [3]:

# Load the saved fold data from the .pkl file
fold_data = torch.load('/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/cross_val_data/fold_data.pt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = '/home/jaiswalsuraj/suraj_work/Brick-Kilns-Project/data/models/'

# Lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold_info in fold_data:
    fold = fold_info['fold']
    if fold == 1 or fold == 2:
        continue
    print("Fold: ", fold)
    X_train = fold_info['X_train']
    y_train = fold_info['y_train']
    X_test = fold_info['X_test']
    y_test = fold_info['y_test']
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoader for training and testing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create and train the model
    #print datatype of trainloader
    print("trainloader datatype: ", train_loader.dataset.tensors[1].dtype)
    print("testloader datatype: ", test_loader.dataset.tensors[0].dtype)
    train_model = EfficientNetClassifier    (
        models.efficientnet_b0, EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
    ).to(device)

    iter_losses, epoch_losses = train_fn(
        train_model,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        lr=3e-4,
        epochs=100,
        verbose=True,
        wandb_log=False,
    )
    torch.save(train_model.state_dict(), save_path + f'efficientnet_train_bang_fold_{fold}.pth')
    
    # Evaluate the model on the test set
    with torch.no_grad():
        pred_classes = train_model.predict_class(
            dataloader=test_loader, batch_size=batch_size, verbose=True
        ).to(device)

    test_labels = y_test.to(device)
    # Calculate and print metrics for each fold
    
    accuracy = accuracy_score(pred_classes,test_labels)
    precision = precision_score( pred_classes,test_labels)
    recall = recall_score( pred_classes,test_labels)
    f1 = f1_score( pred_classes,test_labels)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\n")
    # Calculate and store metrics for each fold
    accuracy_list.append(accuracy_score( pred_classes,test_labels))
    precision_list.append(precision_score( pred_classes,test_labels))
    recall_list.append(recall_score( pred_classes,test_labels))
    f1_list.append(f1_score( pred_classes,test_labels))

# Calculate and print the mean of metrics across all folds
print("Mean Accuracy: ", sum(accuracy_list) / len(accuracy_list))
print("Mean Precision: ", sum(precision_list) / len(precision_list))
print("Mean Recall: ", sum(recall_list) / len(recall_list))
print("Mean F1: ", sum(f1_list) / len(f1_list))


Fold:  3
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


/home/jaiswalsuraj/miniconda3/envs/torch_gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 25/25 [00:02<00:00,  9.03it/s]


Accuracy: 0.9857
Precision: 0.9012
Recall: 0.8821
F1 Score: 0.8915


Fold:  4
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


100%|██████████| 25/25 [00:02<00:00,  9.56it/s]


Accuracy: 0.9870
Precision: 0.8851
Recall: 0.9247
F1 Score: 0.9045


Mean Accuracy:  tensor(0.9864, device='cuda:0')
Mean Precision:  tensor(0.8932, device='cuda:0')
Mean Recall:  tensor(0.9034, device='cuda:0')
Mean F1:  tensor(0.8980, device='cuda:0')


The above model is trained on bangladesh and predicted on test set of bangladesh.

Now we will predict the test set of delhi using the above model.